In [1]:
# from pymongo import MongoClient
import pandas as pd
import requests
from pprint import pprint

In [4]:
# client = MongoClient()
# db = client['meetup']

In [2]:
API_KEY = '6cc211017f64141072497d1766550'
BASE_URL = 'https://api.meetup.com'

In [3]:
# get a listing of all meetup events hosted by DE-ProD-SF

def get_events(urlname):
    
    events_url = BASE_URL + '/' + urlname + '/events'
    params = {'sign':'true','key': API_KEY, 'status': 'past'}
    response = requests.get(events_url, params = params)
    events = response.json()
    
    return events

urlname = 'DE-ProD-SF'
get_events(urlname)

[{u'created': 1455863910000,
  u'description': u"<p>Exercise your critical thinking with a data eng challenge problem! Hear how others would solve the problem, what trade-offs they consider, and voice your opinion / ask questions in a non-threatening atmosphere. Like a journal club, but minus reading boring papers, and plus an exciting problem :-)</p> <p>I'll send out the challenge problem by email to the RSVP'd group 2 days before the event.</p> <p>By the way, employers rank whiteboarding &amp; critical thinking skills as one of their top priorities in hiring data engineers. Practice makes perfect!</p> <p>See you soon,<br/>Aaron</p> ",
  u'group': {u'created': 1441761756000,
   u'id': 18913207,
   u'join_mode': u'open',
   u'lat': 37.779998779296875,
   u'lon': -122.41999816894531,
   u'name': u'Data Engineering Professional Development San Francisco',
   u'urlname': u'DE-ProD-SF',
   u'who': u'Professional Developers'},
  u'id': u'228968660',
  u'link': u'http://www.meetup.com/DE-Pro

In [4]:
#query all event ids and descriptions

def query_events():
    
    event_id = []
    event_description= []
    for event in get_events(urlname):
        event_id.append(event['id'])
        event_description.append(event['description'])
 
    return event_id, event_description

# events_ids = [''.join(event['id']) for event in events]
# print events_ids

# events_descriptions = [event['description'] for event in events]
# print events_descriptions

event_id = query_events()[0]
event_description = query_events()[1]

df_events= pd.DataFrame([event_id, event_description]).T
df_events.columns = ['event_id', 'event_description']
df_events

,event_id,event_description
0,228968660,<p>Exercise your critical thinking with a data...
1,228648803,"<p><b>*Note</b>, expedite your check in at Gal..."


In [9]:
query_events()[0]

[u'228968660', u'228648803']

In [23]:
#get all rsvps for each event

member_id = []
event_id = []

for i in query_events()[0]:
    rsvps_url = BASE_URL + '/' + urlname +'/events/' + i + '/rsvps'
    params = {'sign':'true','key': API_KEY}
    response = requests.get(rsvps_url, params = params)
    rsvps = response.json()
    
    for rsvp in rsvps:
        member_id.append(rsvp['member']['id'])
        event_id.append(rsvp['event']['id'])
               
df_rsvps = pd.DataFrame([member_id, event_id]).T
df_rsvps.columns = ['member_id', 'event_id']
df_rsvps


,member_id,event_id
0,87429312,228968660
1,149624662,228968660
2,129647262,228968660
3,132426572,228968660
4,67449572,228968660
5,111657512,228968660
6,198433955,228968660
7,11847383,228968660
8,10048695,228968660
9,14704591,228968660


In [35]:
# join df_events and df_rsvps
df = pd.merge(df_events, df_rsvps, on = 'event_id')
df['rsvp'] = 1
df
# df2 = df.groupby(member_id).mean()
# df2

,event_id,event_description,member_id,rsvp
0,228968660,<p>Exercise your critical thinking with a data...,87429312,1
1,228968660,<p>Exercise your critical thinking with a data...,149624662,1
2,228968660,<p>Exercise your critical thinking with a data...,129647262,1
3,228968660,<p>Exercise your critical thinking with a data...,132426572,1
4,228968660,<p>Exercise your critical thinking with a data...,67449572,1
5,228968660,<p>Exercise your critical thinking with a data...,111657512,1
6,228968660,<p>Exercise your critical thinking with a data...,198433955,1
7,228968660,<p>Exercise your critical thinking with a data...,11847383,1
8,228968660,<p>Exercise your critical thinking with a data...,10048695,1
9,228968660,<p>Exercise your critical thinking with a data...,14704591,1


In [34]:
# df = pd.DataFrame([member_id, event_id, event_description])
# df.columns = ['member_id', 'event_id', 'event_description']

In [41]:
# get all members from a group
members_url = BASE_URL + '/2/members' 
params = {'sign':'true','key': API_KEY, 'group_id': '18514297'}
response = requests.get(members_url, params = params)
members = response.json()
pprint (members)

{u'meta': {u'count': 200,
           u'description': u'API method for accessing members of Meetup Groups',
           u'id': u'',
           u'lat': u'',
           u'link': u'https://api.meetup.com/2/members',
           u'lon': u'',
           u'method': u'Members',
           u'next': u'https://api.meetup.com/2/members?offset=1&sign=True&format=json&group_id=18514297&page=200&key=6cc211017f64141072497d1766550&order=name',
           u'signed_url': u'https://api.meetup.com/2/members?offset=0&format=json&group_id=18514297&page=200&order=name&sig_id=192752466&sig=049d6dda57229871026fd2287052f4ce3e0dbe8c',
           u'title': u'Meetup Members v2',
           u'total_count': 1162,
           u'updated': 1461773697000,
           u'url': u'https://api.meetup.com/2/members?offset=0&sign=True&format=json&group_id=18514297&page=200&key=6cc211017f64141072497d1766550&order=name'},
 u'results': [{u'bio': u'Data Science Fellow at Galvanize',
               u'city': u'San Francisco',
           

In [40]:
members[]

KeyError: 'id'